In [ ]:
import heapq
from collections import defaultdict
from dataclasses import dataclass

In [ ]:
@dataclass
class Order:
    id: int
    side: str
    price: float
    qty: float
    ts: int


class OrderBook:
    def __init__(self):
        self.bids = []  # (-price, ts, order)
        self.asks = []  # (price, ts, order)
        self.clock = 0

    def _next_ts(self):
        self.clock += 1
        return self.clock

    def add_order(self, side, price, qty):
        ts = self._next_ts()
        order = Order(ts, side, price, qty, ts)

        if side == "buy":
            heapq.heappush(self.bids, (-price, ts, order))
        else:
            heapq.heappush(self.asks, (price, ts, order))

        return self._match()

    def _best_bid(self):
        return self.bids[0][2] if self.bids else None

    def _best_ask(self):
        return self.asks[0][2] if self.asks else None

    def _match(self):
        trades = []

        while self.bids and self.asks:
            bid = self._best_bid()
            ask = self._best_ask()

            if bid.price < ask.price:
                break

            qty = min(bid.qty, ask.qty)
            price = ask.price  # maker price convention

            trades.append((price, qty, bid.id, ask.id))

            bid.qty -= qty
            ask.qty -= qty

            if bid.qty == 0:
                heapq.heappop(self.bids)
            if ask.qty == 0:
                heapq.heappop(self.asks)

        return trades

    def depth(self, levels=5):
        bid_levels = defaultdict(float)
        ask_levels = defaultdict(float)

        for p, _, o in self.bids:
            bid_levels[-p] += o.qty
        for p, _, o in self.asks:
            ask_levels[p] += o.qty

        bid_view = sorted(bid_levels.items(), key=lambda x: -x[0])[:levels]
        ask_view = sorted(ask_levels.items(), key=lambda x: x[0])[:levels]

        return bid_view, ask_view

    def print_depth(self, levels=5):
        bids, asks = self.depth(levels)

        print("\nORDER BOOK (Top 5)")
        print("ASKS:")
        for p, q in asks:
            print(f"  {p:.2f} x {q}")

        print("BIDS:")
        for p, q in bids:
            print(f"  {p:.2f} x {q}")

In [4]:
def run_stream():
    ob = OrderBook()
    trades = []

    stream = [
        ("buy", 100, 5),
        ("buy", 99, 3),
        ("sell", 101, 4),
        ("sell", 100, 2),
        ("sell", 99, 10),
        ("buy", 101, 6),
    ]

    for side, price, qty in stream:
        t = ob.add_order(side, price, qty)
        trades.extend(t)

    ob.print_depth(5)
    return trades


print("RUN 1")
t1 = run_stream()

print("\nRUN 2")
t2 = run_stream()

print("\nTrades identical?", t1 == t2)
print("\nTrades:")
for t in t1:
    print(t)


RUN 1

ORDER BOOK (Top 5 Levels)
ASKS:
  101.00 x 2.0
BIDS:

RUN 2

ORDER BOOK (Top 5 Levels)
ASKS:
  101.00 x 2.0
BIDS:

Trades identical? True

Trades:
(100, 2, 1, 4)
(99, 3, 1, 5)
(99, 3, 2, 5)
(99, 4, 6, 5)
(101, 2, 6, 3)
